In [5]:
import numpy as np
import gym
from types import SimpleNamespace

In [6]:
const = SimpleNamespace()
const.mu = 3.986e14
const.Re = 6037.1e3
sat = SimpleNamespace()
sat.J = np.diag(np.array([4, 5, 3]))
sat.J_inv = np.linalg.inv(sat.J)
sat.altitude = 600e3
sat.mean_motion = np.sqrt(const.mu / (const.Re + sat.altitude)**3.)


class TorqueDynamics(gym.Env):
  observation_space = gym.spaces.Box(-1, 1, shape=(10,))

  
  def __init__(self, dt, q_req, add_9=0.25):
    self.state = None
    self.dt = dt
    self.q_req = q_req
    self.q_req_conj = quat_conjugate(self.q_req)
    self.w_req = np.zeros(3)
    self.history = []
    self.t = []
    self.action_space = self.init_actions()
    self.q_prev = None
    self.add_9 = add_9

  @staticmethod
  def init_actions():
    a = np.linspace(-1, 1, 21)
    a1 = a/10
    a2 = a/100
    a = np.concatenate((a, a1, a2))
    a = np.unique(a.round(10))
    a = a[a>=-0.5]
    a = a[a<=0.5]

    s = a.shape
    a = np.vstack((a, np.zeros(s)))
    a = np.vstack((a, np.zeros(s)))
    a = a.T
    aroll1 = np.roll(a, 1)
    aroll2 = np.roll(a, 2)
    a = np.concatenate((a, aroll1))
    a = np.concatenate((a, aroll2))
    a = np.unique(a, axis=0)
    return a

  def reset(self, state=None):
    if state is not None:
      self.state = state
    else:
      self.state = self.observation_space.sample()
      phi = self.state[0] * np.pi 
      self.state[0] = np.cos(phi / 2)
      self.state[1:4] = normalize(self.state[1:4]) * np.sin(phi / 2)
      self.state[4:] = 0
    self.history = [self.state]
    self.t = [0]
    self.q_prev = self.state[:4]
    self.multiplier = 1
    return self.state

  @staticmethod
  def r_a(phi, q_current, q_prev):
    return np.exp(2-phi) if q_current > q_prev else np.exp(2-phi)/2


  def r_t(self, reward, phi):
    return reward + 9 if phi <= self.add_9 else reward

  def step(self, action):
    t0 = 0
    tf = self.dt
    x_0 = self.state
    if isinstance(action, int) or isinstance(action, np.int64):
      action = self.action_space[action].copy()
    
    action *= self.multiplier
    
    sol = solve_ivp(lambda t, x: rhs(t, x, sat, action), (t0,tf), x_0)#, t_eval=t_eval)
    x = sol.y.T
    t = sol.t[1:]
    observation = x[-1]
    observations = normalize(observation)
    time_ = self.t[-1] + self.dt
    self.state = observation
    self.history.append(observation)
    self.t.append(time_)

    # calculating rewards:
    q_current = observation[:4]
    q_error = quat_product(self.q_req_conj, q_current)
    q_error = np.clip(q_error, -1, 1)
    w_current = observation[4:7]
    #print('q_error', q_error)
    phi = 2*np.arccos(q_error[0])
    r_inter = self.r_a(phi, q_current[0], self.q_prev[0])
    r1 = self.r_t(r_inter, phi)
    # r2 = -np.sum(np.abs(observation[4:7]))
    # Qreward = np.exp(-0.1 * np.linalg.norm(q_current - self.q_req))
    # Wreward = np.exp(-0.1 * np.linalg.norm(w_current - self.w_req))
    # reward = Qreward * Wreward
    # reward = self.r_t(reward, phi)
    reward = r_inter #r1 #+ 10 * r2
    #print('rewards', r1, r2)
    
    self.multiplier = 1 if phi > np.pi/8 else np.sin(4*phi)
    
    self.q_prev = q_current

    q_req_ext = np.concatenate([self.q_req, self.w_req])
    done = np.linalg.norm(observation[:7] - q_req_ext) < 1e-4
    
    info = dict()
    info['x'] = self.history
    info['t'] = self.t
    return observation, reward, done, info
  
  def render(self):
    pass